# Getting started - Requirements
Python (🐍):  This assumes that you have a python environment installed. If you haven't, we recommend installing [Anaconda](https://docs.anaconda.com/anaconda/install/).

FPsim (♀️): This tutorial uses [FPsim v2.0](https://github.com/fpsim/fpsim/tree/rc2.0-methodtime) (still under development). 

To install, you first need to clone or download the a copy of the source code from https://github.com/starsimhub/typhoidsim
 
```
git clone -b rc2.0-methodtime --single-branch https://github.com/fpsim/fpsim.git
cd fpsim
pip install -e .
```

If it worked, you should be able to import typhoidsim with `import fpsim as fp`.

## Interventions - what if we want a percentage of the eligible population? 

You can use the same mechanism we saw in the previous tutiorial (T4) to the eleigibility criteria for yout intervention. An example of a more complex eligibility function could be:

_Select a percentage of urban women aged 20-25 who have 1+ children, are in the lowest wealth quintile, and have intent to use contraception_



In [ ]:
import fpsim as fp
import sciris as sc

In [ ]:
def eligibility_criteria(sim):
    min_age = 20
    max_age = 25
    # Select all women that meet our criteria
    is_eligible = ((sim.people.is_female) &
                   (sim.people.alive) &
                   (sim.people.urban == True)  &
                   (sim.people.age >= min_age) &
                   (sim.people.age <  max_age) &
                   (sim.people.parity >= 1) & 
                   (sim.people.wealthquintile == 1) & 
                   (sim.people.intent_to_use == True))
    
    return is_eligible

In [ ]:
change_intent = fp.change_people_state('intent_to_use', 
                                       new_val=True, 
                                       eligibility=eligibility_criteria, 
                                       year=2005.0, 
                                       prop=0.5) # only 50% of women who meet the criteria are selected for the intervention

In [ ]:
pars = dict(
    n_agents   = 10_000,
    location   = 'kenya',
    start_year = 2000, 
    end_year   = 2012,
    exposure_factor = 1.0  # Overall scale factor on probability of becoming pregnant
)


In [ ]:
s1 = fp.Sim(pars=pars, label='Baseline')
s2 = fp.Sim(pars=pars, interventions=change_intent, label='Increased intent')
simlist = sc.autolist([s1, s2])
msim = fp.MultiSim(sims=simlist)
msim.run(serial=True, die=False)

In [ ]:
msim.plot(to_plot='intent');